In [29]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [30]:
from Utils.recsys2022DataReader import createBumpURM
from Utils.recsys2022DataReader import createSmallICM
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.Hybrid.MergedHybridRecommender import MergedHybridRecommender
from optuna.samplers import TPESampler
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
import optuna as op

In [31]:
URM = createBumpURM()
ICM = createSmallICM()

C:\Users\Mike\PycharmProjects\recsys-challenge-2022-Pancini-Vitali\Utils\recsys2022DataReader.py:16: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(urmPath)


In [32]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM, 0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, 0.85)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 872 ( 2.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 1555 ( 3.7%) Users that have less than 1 test interactions


In [ ]:
ItemKNNCF_recommender = ItemKNNCFRecommender(URM_train)
ItemKNNCBF_recommender = ItemKNNCBFRecommender(URM_train, ICM)

In [ ]:
def objective(trial):

    topKCF = trial.suggest_int("topKCF", 300, 400)
    shrinkCF = trial.suggest_float("shrinkCF", 10, 100)
    topKCBF = trial.suggest_int("topKCBF", 300, 400)
    shrinkCBF = trial.suggest_float("shrinkCBF", 10, 100)
    alpha = trial.suggest_float("alpha", 0.1, 0.9)

    ItemKNNCF_recommender.fit(topK=topKCF, shrink=shrinkCF)
    ItemKNNCBF_recommender.fit(topK=topKCBF, shrink=shrinkCBF)

    recommender = MergedHybridRecommender(URM_train, recommender1=ItemKNNCF_recommender, recommender2=ItemKNNCBF_recommender, verbose=False)
    recommender.fit(alpha=alpha)

    result, _ = evaluator_validation.evaluateRecommender(recommender)

    return result.loc[10]['MAP']

In [33]:
study = op.create_study(direction='maximize', sampler=TPESampler(multivariate=True))
study.optimize(objective, n_trials=10)

ItemKNNCBFRecommender: ICM Detected 4877 (19.9%) items with no features.
Similarity column 24507 (100.0%), 6543.54 column/sec. Elapsed time 3.75 sec
Compute_Similarity: detected dense matrix
Similarity column 24507 (100.0%), 2214.74 column/sec. Elapsed time 11.07 sec


In [ ]:
ItemKNNCF_recommender.fit(topK=study.best_params['topKCF'], shrink=study.best_params['shrinkCF'])
ItemKNNCBF_recommender.fit(topK=study.best_params['topKCBF'], shrink=study.best_params['shrinkCBF'])

recommender = MergedHybridRecommender(URM_train, recommender1=ItemKNNCF_recommender, recommender2=ItemKNNCBF_recommender, verbose=False)
recommender.fit(alpha=study.best_params['alpha'])

In [34]:
result, _ = evaluator_test.evaluateRecommender(recommender)
result

EvaluatorHoldout: Processed 40757 (100.0%) in 41.05 sec. Users per second: 993


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.042596,0.071097,0.066523,0.019802,0.032684,0.140396,0.072498,0.051936,0.298893,0.165825,...,0.979053,0.292633,0.979053,0.058722,9.197093,0.992407,0.116006,0.681632,4.009202,0.298335
